In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys


In [ ]:
df1 = pd.read_csv('../data_files/raw/modelslant_pilot_llm_responses.csv')
df2 = pd.read_csv('../data_files/raw/modelslant_questions_endpoints.csv')

In [ ]:
df = df1.merge(df2, on='Topic', how='left')

In [ ]:
df

In [ ]:
# Process the dataframe into the desired format
processed_data = []

# Get unique question IDs
question_ids = df['Question'].unique()

for i, question in enumerate(question_ids):
    # Filter rows for this question
    question_df = df[df['Question'] == question]
    option1 = question_df['Endpoint 1'].values[0]
    option2 = question_df['Endpoint 2'].values[0]

    # Add a row just for the question
    question_entry = {
        "id": f"{i+1}_select",
        "question": question,
        "text": f"[Question] {question}".replace("\n", "<br>") + f"<br> <br> [Options] <br> 1. {option1} <br> 2. {option2}",
        "option1": option1,
        "option2": option2,
        "model": None,
        "response_length": None
    }
    processed_data.append(question_entry)
    
    # Add a row just for the question
    question_entry = {
        "id": f"{i+1}_question",
        "question": question,
        "text": f"[Question] {question}".replace("\n", "<br>"),
        "option1": None,
        "option2": None,
        "model": None,
        "response_length": None
    }
    processed_data.append(question_entry)
    
    question_df = question_df.sample(frac=1).reset_index(drop=True)
    # Add entries for each model's response to this question
    for _, row in question_df.iterrows():
        model = row['Model']
        response = row['Response']
        entry = {
            "id": f"{i+1}_{model}",
            "question": question,
            "model": model,
            "text": f"Question: {question} <br> <hr> LLM generated response: <br> <br> {response}".replace("\n", "<br>"),
            "option1": None,
            "option2": None,
            "response_length": len(response)
        }
        processed_data.append(entry)

# Convert the processed data into a DataFrame
processed_df = pd.DataFrame(processed_data)


In [ ]:
processed_df.text.values[2]

In [ ]:
processed_df

In [ ]:
processed_df.to_csv('../data_files/processed/pilot_modelslant_questions_with_responses.csv', index=False)